In [1]:
# 不需要使用accelerator 因为我们只有一个gpu

In [2]:
!git clone https://github.com/huggingface/diffusers
%cd diffusers
!pip install .

fatal: destination path 'diffusers' already exists and is not an empty directory.
/content/diffusers
Processing /content/diffusers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.24.0.dev0-py3-none-any.whl size=1729690 sha256=7c04e9cf551459f175f5ef023647e6119bb681fabb71ad2973756e8265054803
  Stored in directory: /tmp/pip-ephem-wheel-cache-_mliq2ho/wheels/95/c5/3b/e1b4269f8a2584de57e75f949a185b48fc4144e9a91fc9965a
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.24.0.dev0
    Uninstalling diffusers-0.24.0.dev0:
      Successfully uninstalled diffusers-0.24.0.dev0


In [3]:
%cd text_to_image

[Errno 2] No such file or directory: 'text_to_image'
/content/diffusers


In [4]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [5]:
! pip install wandb

In [6]:
import wandb
wandb.login()

wandb: Currently logged in as: xiaochy (cs182_xiaochy). Use `wandb login --relogin` to force relogin


True

In [7]:
import argparse
import logging
import math
import os
import random
import shutil
from pathlib import Path

import datasets
import numpy as np
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
import transformers
from accelerate import Accelerator
# 如何使用get_logger获得log信息？
from accelerate.logging import get_logger
from accelerate.utils import ProjectConfiguration, set_seed
from datasets import load_dataset
from huggingface_hub import create_repo, upload_folder
from packaging import version
from torchvision import transforms
# tqdm 是一个用于在 Python 中显示进度条的库，它通常在循环中使用，以便用户可以实时看到代码的执行进度。
from tqdm.auto import tqdm
# CLIPTextModel： 它是一个用于处理文本的预训练模型，可以接收输入文本并生成对应的文本表示。你可以使用这个模型来提取文本特征或进行文本相关的任务
# CLIPTokenizer： 用于对输入文本进行标记化（tokenization）的 CLIP 模型的标记器
from transformers import CLIPTextModel, CLIPTokenizer

import diffusers
from diffusers import AutoencoderKL, DDPMScheduler, DiffusionPipeline, UNet2DConditionModel
from diffusers.models.lora import LoRALinearLayer
from diffusers.optimization import get_scheduler
from diffusers.training_utils import compute_snr
from diffusers.utils import check_min_version, is_wandb_available
from diffusers.utils.import_utils import is_xformers_available


# Will error if the minimal version of diffusers is not installed. Remove at your own risks.
check_min_version("0.24.0.dev0")

logger = get_logger(__name__, log_level="INFO")

In [8]:
# TODO: This function should be removed once training scripts are rewritten in PEFT
def text_encoder_lora_state_dict(text_encoder):
    state_dict = {}

    def text_encoder_attn_modules(text_encoder):
        from transformers import CLIPTextModel, CLIPTextModelWithProjection

        attn_modules = []

        if isinstance(text_encoder, (CLIPTextModel, CLIPTextModelWithProjection)):
            for i, layer in enumerate(text_encoder.text_model.encoder.layers):
                name = f"text_model.encoder.layers.{i}.self_attn"
                mod = layer.self_attn
                attn_modules.append((name, mod))

        return attn_modules

    for name, module in text_encoder_attn_modules(text_encoder):
        for k, v in module.q_proj.lora_linear_layer.state_dict().items():
            state_dict[f"{name}.q_proj.lora_linear_layer.{k}"] = v

        for k, v in module.k_proj.lora_linear_layer.state_dict().items():
            state_dict[f"{name}.k_proj.lora_linear_layer.{k}"] = v

        for k, v in module.v_proj.lora_linear_layer.state_dict().items():
            state_dict[f"{name}.v_proj.lora_linear_layer.{k}"] = v

        for k, v in module.out_proj.lora_linear_layer.state_dict().items():
            state_dict[f"{name}.out_proj.lora_linear_layer.{k}"] = v

    return state_dict


def save_model_card(repo_id: str, images=None, base_model=str, dataset_name=str, repo_folder=None):
    img_str = ""
    for i, image in enumerate(images):
        image.save(os.path.join(repo_folder, f"image_{i}.png"))
        img_str += f"![img_{i}](./image_{i}.png)\n"

    yaml = f"""
---
license: creativeml-openrail-m
base_model: {base_model}
tags:
- stable-diffusion
- stable-diffusion-diffusers
- text-to-image
- diffusers
- lora
inference: true
---
    """
    model_card = f"""
# LoRA text2image fine-tuning - {repo_id}
These are LoRA adaption weights for {base_model}. The weights were fine-tuned on the {dataset_name} dataset. You can find some example images in the following. \n
{img_str}
"""
    with open(os.path.join(repo_folder, "README.md"), "w") as f:
        f.write(yaml + model_card)

In [9]:
pretrained_model_name_or_path = "runwayml/stable-diffusion-v1-5"
dataset_name = "lambdalabs/pokemon-blip-captions"
revision = None
train_data_dir = None
variant = None
dataset_config_name = None
image_column = "image"
caption_column = "text"
validation_prompt = "Totoro"
num_validation_images = 4
validation_epochs = 1
max_train_samples = None
output_dir = "/content/drive/MyDrive/huggingface_diffuser/model"
cache_dir = None
resolution = 512
center_crop = False
random_flip = False
train_batch_size = 1
gradient_accumulation_steps = 1
max_train_steps = 15000
num_train_epochs = 100
gradient_checkpointing = False
learning_rate = 1e-4
scale_lr = False
lr_scheduler = "cosine"
lr_warmup_steps = 0
snr_gamma = None
use_8bit_adam = False
allow_tf32 = False
dataloader_num_workers = 2
adam_beta1 = 0.9
adam_beta2 = 0.999
adam_weight_decay = 1e-2
adam_epsilon = 1e-08
max_grad_norm = 1
push_to_hub = False
hub_model_id = None
prediction_type = None
mixed_precision = "fp16"
report_to = "wandb" # tensorborad
local_rank = -1
checkpointing_steps = 500
checkpoints_total_limit = None
resume_from_checkpoint = None
enable_xformers_memory_efficient_attention = False
noise_offset = 0
logging_dir = "logs"
# lora rank
rank = 4
seed = 1337
DATASET_NAME_MAPPING = {
    "lambdalabs/pokemon-blip-captions": ("image", "text"),
}

In [10]:
!pip install wandb

In [11]:
logging_dir = Path(output_dir, logging_dir)
# accelerator_project_config = ProjectConfiguration(project_dir=output_dir, logging_dir=logging_dir)


# accelerator = Accelerator(
#         gradient_accumulation_steps=gradient_accumulation_steps,
#         mixed_precision=mixed_precision,
#         log_with=report_to,
#         project_config=accelerator_project_config,
#     )

# if report_to == "wandb":
#   if not is_wandb_available():
#       raise ImportError("Make sure to install wandb if you want to use it for logging during training.")
#   import wandb

# Make one log on every process with the configuration for debugging.
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

# logger.info(accelerator.state, main_process_only=False)
# if accelerator.is_local_main_process:
#     datasets.utils.logging.set_verbosity_warning()
#     transformers.utils.logging.set_verbosity_warning()
#     diffusers.utils.logging.set_verbosity_info()
# else:
#     datasets.utils.logging.set_verbosity_error()
#     transformers.utils.logging.set_verbosity_error()
#     diffusers.utils.logging.set_verbosity_error()

set_seed(seed)


# Load scheduler, tokenizer and models.
# 将pretrained diffusion model 的各个部分都load进来
noise_scheduler = DDPMScheduler.from_pretrained(pretrained_model_name_or_path, subfolder="scheduler")
tokenizer = CLIPTokenizer.from_pretrained(
    pretrained_model_name_or_path, subfolder="tokenizer", revision=revision
)
text_encoder = CLIPTextModel.from_pretrained(
    pretrained_model_name_or_path, subfolder="text_encoder", revision=revision
)
vae = AutoencoderKL.from_pretrained(
    pretrained_model_name_or_path, subfolder="vae", revision=revision, variant=variant
)
unet = UNet2DConditionModel.from_pretrained(
    pretrained_model_name_or_path, subfolder="unet", revision=revision, variant=variant
)
# freeze parameters of models to save more memory
# 即我们不需要改Model原先的parameters,我们只需要改lora层的parameters
unet.requires_grad_(False)
vae.requires_grad_(False)
text_encoder.requires_grad_(False)


CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 768)
      (position_embedding): Embedding(77, 768)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e

In [12]:
#For mixed precision training we cast all non-trainable weigths (vae, non-lora text_encoder and non-lora unet) to half-precision
#as these weights are only used for inference, keeping weights in full precision is not required.

# weight_dtype = torch.float32
# if accelerator.mixed_precision == "fp16":
#     weight_dtype = torch.float16
# elif accelerator.mixed_precision == "bf16":
#     weight_dtype = torch.bfloat16

# Move unet, vae and text_encoder to device and cast to weight_dtype
# unet.to(accelerator.device, dtype=weight_dtype)
# vae.to(accelerator.device, dtype=weight_dtype)
# text_encoder.to(accelerator.device, dtype=weight_dtype)

In [13]:
# 开始finetune!!! 调整的是unet中的weights!
# now we will add new LoRA weights to the attention layers
# It's important to realize here how many attention weights will be added and of which sizes
# The sizes of the attention layers consist only of two different variables:

# 1) - the "hidden_size", which is increased according to `unet.config.block_out_channels`.
# 2) - the "cross attention size", which is set to `unet.config.cross_attention_dim`.

# Let's first see how many attention processors we will have to set.
# For Stable Diffusion, it should be equal to:
# - down blocks (2x attention layers) * (2x transformer layers) * (3x down blocks) = 12
# - mid blocks (2x attention layers) * (1x transformer layers) * (1x mid blocks) = 2
# - up blocks (2x attention layers) * (3x transformer layers) * (3x down blocks) = 18
# => 32 layers

# Set correct lora layers
unet_lora_parameters = []
# attn_processor_name is the key
# attn_processor is the value
for attn_processor_name, attn_processor in unet.attn_processors.items():
    # Parse the attention module.
    attn_module = unet
    for n in attn_processor_name.split(".")[:-1]:
        attn_module = getattr(attn_module, n)

    # Set the `lora_layer` attribute of the attention-related matrices.
    attn_module.to_q.set_lora_layer(
        LoRALinearLayer(
            in_features=attn_module.to_q.in_features, out_features=attn_module.to_q.out_features, rank=rank
        )
    )
    attn_module.to_k.set_lora_layer(
        LoRALinearLayer(
            in_features=attn_module.to_k.in_features, out_features=attn_module.to_k.out_features, rank=rank
        )
    )

    attn_module.to_v.set_lora_layer(
        LoRALinearLayer(
            in_features=attn_module.to_v.in_features, out_features=attn_module.to_v.out_features, rank=rank
        )
    )
    attn_module.to_out[0].set_lora_layer(
        LoRALinearLayer(
            in_features=attn_module.to_out[0].in_features,
            out_features=attn_module.to_out[0].out_features,
            rank=rank,
        )
    )

    # Accumulate the LoRA params to optimize.
    unet_lora_parameters.extend(attn_module.to_q.lora_layer.parameters())
    unet_lora_parameters.extend(attn_module.to_k.lora_layer.parameters())
    unet_lora_parameters.extend(attn_module.to_v.lora_layer.parameters())
    unet_lora_parameters.extend(attn_module.to_out[0].lora_layer.parameters())



optimizer_cls = torch.optim.AdamW

optimizer = optimizer_cls(
    unet_lora_parameters,
    lr=learning_rate,
    betas=(adam_beta1,adam_beta2),
    weight_decay=adam_weight_decay,
    eps=adam_epsilon,
)

# Get the datasets: you can either provide your own training and evaluation files (see below)
# or specify a Dataset from the hub (the dataset will be downloaded automatically from the datasets Hub).

# In distributed training, the load_dataset function guarantees that only one local process can concurrently
# download the dataset.
if dataset_name is not None:
    # Downloading and loading a dataset from the hub.
    dataset = load_dataset(
        dataset_name,
        dataset_config_name,
        cache_dir=cache_dir,
        data_dir=train_data_dir,
    )
else:
    data_files = {}
    if train_data_dir is not None:
        data_files["train"] = os.path.join(train_data_dir, "**")
    dataset = load_dataset(
        "imagefolder",
        data_files=data_files,
        cache_dir=cache_dir,
    )
    # See more about loading custom images at
    # https://huggingface.co/docs/datasets/v2.4.0/en/image_load#imagefolder

# Preprocessing the datasets.
# We need to tokenize inputs and targets.
column_names = dataset["train"].column_names

# 6. Get the column names for input/target.
dataset_columns = DATASET_NAME_MAPPING.get(dataset_name, None) # 这里前面DATASET_NAME_MAPPING是不是要改一下，否则会出错
if image_column is None:
    image_column = dataset_columns[0] if dataset_columns is not None else column_names[0]
else:
    image_column = image_column
    if image_column not in column_names:
        raise ValueError(
            f"--image_column' value '{image_column}' needs to be one of: {', '.join(column_names)}"
        )
if caption_column is None:
    caption_column = dataset_columns[1] if dataset_columns is not None else column_names[1]
else:
    caption_column = caption_column
    if caption_column not in column_names:
        raise ValueError(
            f"--caption_column' value '{caption_column}' needs to be one of: {', '.join(column_names)}"
        )

In [14]:
# Preprocessing the datasets.
# We need to tokenize input captions and transform the images.
def tokenize_captions(examples, is_train=True):
    captions = []
    for caption in examples[caption_column]:
        if isinstance(caption, str):
            captions.append(caption)
        elif isinstance(caption, (list, np.ndarray)):
            # take a random caption if there are multiple
            captions.append(random.choice(caption) if is_train else caption[0])
        else:
            raise ValueError(
                f"Caption column `{caption_column}` should contain either strings or lists of strings."
            )
    inputs = tokenizer(
        captions, max_length=tokenizer.model_max_length, padding="max_length", truncation=True, return_tensors="pt"
    )
    return inputs.input_ids

# Preprocessing the datasets.
train_transforms = transforms.Compose(
    [
        transforms.Resize(resolution, interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.CenterCrop(resolution) if center_crop else transforms.RandomCrop(resolution),
        transforms.RandomHorizontalFlip() if random_flip else transforms.Lambda(lambda x: x),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

def preprocess_train(examples):
    images = [image.convert("RGB") for image in examples[image_column]]
    examples["pixel_values"] = [train_transforms(image) for image in images]
    examples["input_ids"] = tokenize_captions(examples)
    return examples

# with accelerator.main_process_first():
if max_train_samples is not None:
    dataset["train"] = dataset["train"].shuffle(seed=seed).select(range(max_train_samples))
# Set the training transforms
train_dataset = dataset["train"].with_transform(preprocess_train)

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()
    input_ids = torch.stack([example["input_ids"] for example in examples])
    return {"pixel_values": pixel_values, "input_ids": input_ids}

# DataLoaders creation:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=collate_fn,
    batch_size=train_batch_size,
    num_workers=dataloader_num_workers,
)

# Scheduler and math around the number of training steps.
overrode_max_train_steps = False
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
if max_train_steps is None:
    max_train_steps = num_train_epochs * num_update_steps_per_epoch
    overrode_max_train_steps = True

lr_scheduler = get_scheduler(
    lr_scheduler,
    optimizer=optimizer,
    # num_warmup_steps=lr_warmup_steps * accelerator.num_processes,
    # num_training_steps=max_train_steps * accelerator.num_processes,
    num_warmup_steps=lr_warmup_steps,
    num_training_steps=max_train_steps,
)

# Prepare everything with our `accelerator`.
# unet_lora_parameters, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
#     unet_lora_parameters, optimizer, train_dataloader, lr_scheduler
# )

# We need to recalculate our total training steps as the size of the training dataloader may have changed.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
if overrode_max_train_steps:
    max_train_steps = num_train_epochs * num_update_steps_per_epoch
# Afterwards we recalculate our number of training epochs
num_train_epochs = math.ceil(max_train_steps / num_update_steps_per_epoch)



In [15]:
trainning_loss = []

In [16]:
wandb.init()

In [ ]:
# Train!
# total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps
total_batch_size = train_batch_size * gradient_accumulation_steps
global_step = 0
first_epoch = 0

# Potentially load in the weights and states from a previous save
if resume_from_checkpoint:
    if resume_from_checkpoint != "latest":
        path = os.path.basename(resume_from_checkpoint)
    else:
        # Get the most recent checkpoint
        dirs = os.listdir(output_dir)
        dirs = [d for d in dirs if d.startswith("checkpoint")]
        dirs = sorted(dirs, key=lambda x: int(x.split("-")[1]))
        path = dirs[-1] if len(dirs) > 0 else None

else:
    initial_global_step = 0

max_train_steps = 3500

progress_bar = tqdm(
    range(0, max_train_steps),
    initial=initial_global_step,
    desc="Steps",
    # Only show the progress bar once on each machine.
    #disable=not accelerator.is_local_main_process,
)
max_epoch = 1 #5
for epoch in range(max_epoch):
    unet.train()
    # train_loss = 0.0
    for step, batch in enumerate(train_dataloader):
      if step > 10:
        break
      # Convert images to latent space
      latents = vae.encode(batch["pixel_values"]).latent_dist.sample()
      latents = latents * vae.config.scaling_factor

      # Sample noise that we'll add to the latents
      noise = torch.randn_like(latents)
      if noise_offset:
          # https://www.crosslabs.org//blog/diffusion-with-offset-noise
          noise += noise_offset * torch.randn(
              (latents.shape[0], latents.shape[1], 1, 1), device=latents.device
          )

      bsz = latents.shape[0]
      # Sample a random timestep for each image
      timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (bsz,), device=latents.device)
      timesteps = timesteps.long()

      # Add noise to the latents according to the noise magnitude at each timestep
      # (this is the forward diffusion process)
      noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

      # Get the text embedding for conditioning
      encoder_hidden_states = text_encoder(batch["input_ids"])[0]

      # Get the target for loss depending on the prediction type
      if prediction_type is not None:
          # set prediction_type of scheduler if defined
          noise_scheduler.register_to_config(prediction_type=prediction_type)

      if noise_scheduler.config.prediction_type == "epsilon":
          target = noise
      elif noise_scheduler.config.prediction_type == "v_prediction":
          target = noise_scheduler.get_velocity(latents, noise, timesteps)
      else:
          raise ValueError(f"Unknown prediction type {noise_scheduler.config.prediction_type}")

      # Predict the noise residual and compute loss
      model_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample
      loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")

      # Backpropagate
      loss.backward()
      trainning_loss.append(loss.item())
      print("loss = "+str(loss.item()))
      #writer.add_scalar('training loss', loss, epoch*len(train_dataloader)+step)
      wandb.log({"loss": loss})

      # unet = unet.to(torch.float32)
      # unet.save_attn_procs(output_dir)
      # # Final inference
      # # Load previous pipeline
      # pipeline = DiffusionPipeline.from_pretrained(
      #     pretrained_model_name_or_path, revision=revision, variant=variant
      # )
      # #torch_dtype=weight_dtype
      # #pipeline = pipeline.to(accelerator.device)
      # # load attention processors
      # pipeline.unet.load_attn_procs(output_dir)

      # # run inference
      # generator = torch.Generator()
      # if seed is not None:
      #     generator = generator.manual_seed(seed)
      # images = []
      # for _ in range(num_validation_images):
      #     images.append(pipeline(validation_prompt, num_inference_steps=30, generator=generator).images[0])
      # wandb.log({"validation": [wandb.Image(image, caption=f"{i}: {validation_prompt}")for i, image in enumerate(images)]})

      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      progress_bar.update(1)


unet = unet.to(torch.float32)
unet.save_attn_procs(output_dir)


# Final inference
# Load previous pipeline
pipeline = DiffusionPipeline.from_pretrained(
    pretrained_model_name_or_path, revision=revision, variant=variant
)
#torch_dtype=weight_dtype
#pipeline = pipeline.to(accelerator.device)

# load attention processors
pipeline.unet.load_attn_procs(output_dir)

# run inference
generator = torch.Generator(device='cuda')
if seed is not None:
    generator = generator.manual_seed(seed)
images = []
for _ in range(num_validation_images):
    images.append(pipeline(validation_prompt, num_inference_steps=30, generator=generator).images[0])
wandb.log({"validation": [wandb.Image(image, caption=f"{i}: {validation_prompt}")for i, image in enumerate(images)]})

# unet = unet.to(torch.float32)
#       unet.save_attn_procs(output_dir)
#       # Final inference
#       # Load previous pipeline
#       pipeline = DiffusionPipeline.from_pretrained(
#           pretrained_model_name_or_path, revision=revision, variant=variant
#       )
#       #torch_dtype=weight_dtype
#       #pipeline = pipeline.to(accelerator.device)
#       # load attention processors
#       pipeline.unet.load_attn_procs(output_dir)

#       # run inference
#       generator = torch.Generator()
#       if seed is not None:
#           generator = generator.manual_seed(seed)
#       images = []
#       for _ in range(num_validation_images):
#           images.append(pipeline(validation_prompt, num_inference_steps=30, generator=generator).images[0])
#       wandb.log({"validation": [wandb.Image(image, caption=f"{i}: {validation_prompt}")for i, image in enumerate(images)]})
# if len(images) != 0:
#   np_images = np.stack([np.asarray(img) for img in images])
#   writer.add_images("test", np_images, dataformats="NHWC")


Steps:   0%|          | 0/3500 [00:00<?, ?it/s]

loss = 0.07594747841358185
loss = 0.010293630883097649
loss = 0.006291042547672987
loss = 0.057483211159706116
loss = 0.004342556931078434
loss = 0.05254446715116501
loss = 0.01985073834657669
loss = 0.06862937659025192
loss = 0.01957985758781433
loss = 0.03688468411564827
loss = 0.14459940791130066


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [ ]:
# Save the lora layers
#accelerator.wait_for_everyone()
#if accelerator.is_main_process:
unet = unet.to(torch.float32)
unet.save_attn_procs(output_dir)


# Final inference
# Load previous pipeline
pipeline = DiffusionPipeline.from_pretrained(
    pretrained_model_name_or_path, revision=revision, variant=variant
)
#torch_dtype=weight_dtype
#pipeline = pipeline.to(accelerator.device)

# load attention processors
pipeline.unet.load_attn_procs(output_dir)

# run inference
generator = torch.Generator(device=accelerator.device)
if seed is not None:
    generator = generator.manual_seed(seed)
images = []
for _ in range(num_validation_images):
    images.append(pipeline(validation_prompt, num_inference_steps=30, generator=generator).images[0])

if accelerator.is_main_process:
    for tracker in accelerator.trackers:
        if len(images) != 0:
            if tracker.name == "tensorboard":
                np_images = np.stack([np.asarray(img) for img in images])
                tracker.writer.add_images("test", np_images, epoch, dataformats="NHWC")
            if tracker.name == "wandb":
                tracker.log(
                    {
                        "test": [
                            wandb.Image(image, caption=f"{i}: {validation_prompt}")
                            for i, image in enumerate(images)
                        ]
                    }
                )

accelerator.end_training()